In [1]:
import numpy as np 
import pandas as pd 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gc
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", 
                       "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", 
                       "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", 
                       "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                       "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                       "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                       "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", 
                       "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                       "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", 
                       "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                       "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                       "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is",
                       "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                       "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", 
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                       "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                       "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", 
                       "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                       "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", 
                       "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                       "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                       "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", 
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                       "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                       "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [3]:
def preprocess(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [4]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [5]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)


def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [6]:
EMBEDDING_FILES = ['../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec',
    '../input/glove840b300dtxt/glove.840B.300d.txt']

train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv', index_col='id')
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', index_col='id')

x_train=preprocess(train["comment_text"])
x_test=preprocess(test["comment_text"])
y_train=preprocess(train["target"])


/opt/conda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
#cleaning contractions
text_list=pd.concat([x_train, x_test])
temp_list = text_list.apply(lambda x: x.lower())
text_list = temp_list.apply(lambda x: clean_contractions(x, contraction_mapping))
num_train_data = y_train.shape[0]
x_train=text_list[:num_train_data]
x_text=text_list[num_train_data:]

In [8]:
maxlen = 220
#limiting max features will decrease the accuracy, we need more vocab
#max_features = 100000
tokenizer = Tokenizer() #filters = ''

print('fitting tokenizer')
tokenizer.fit_on_texts(list(x_train) + list(x_test))

word_index = tokenizer.word_index

x_train = tokenizer.texts_to_sequences(list(x_train))
y_train = train['target'].values
x_test = tokenizer.texts_to_sequences(list(x_test))


fitting tokenizer


In [9]:
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


del tokenizer
gc.collect()

0

In [10]:
embedding_matrix = np.concatenate(
    [build_matrix(word_index, f) for f in EMBEDDING_FILES], axis=-1)

In [11]:
#embedding_matrix = build_matrix(word_index, embeddings_index)
print(*embedding_matrix.shape)

327576 600


In [12]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [13]:
import keras.layers as L
from keras.models import Model

LSTM_UNITS=128

def build_model(embedding_matrix, verbose = False, compile = True):
    """
    model alchitecture:
    embedding layer
    droput layer
    2 * bidirectional LSTM layers uxing CuDNNLSTM
    an attention layer
    2 * pooling layers
    a dense layer using sigmoid as the activation
    """
    
    embedding_layer = L.Embedding(*embedding_matrix.shape,
                                weights=[embedding_matrix],
                                input_length=maxlen,
                                trainable=False)
    sequence_input = L.Input(shape=(maxlen,), dtype='int32')
    x = embedding_layer(sequence_input)
    x = L.SpatialDropout1D(0.3)(x)
    x = L.Bidirectional(L.CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    x = L.Bidirectional(L.CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    att = Attention(maxlen)(x)
    avg_pool = L.GlobalAveragePooling1D()(x)
    max_pool = L.GlobalMaxPooling1D()(x)

    x = L.concatenate([att, avg_pool, max_pool])

    preds = L.Dense(1, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    
    if verbose:
        model.summary()
    if compile:
        model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['acc'])
        
    return model


In [14]:
from sklearn.model_selection import KFold
import keras.backend as K
import numpy as np

BATCH_SIZE = 2048
NUM_EPOCHS = 2

splits = list(KFold(n_splits=4).split(x_train,y_train))
train_preds = np.zeros((x_train.shape[0]))
test_preds = np.zeros((x_test.shape[0]))
for fold in [0,1,2,3]:
    K.clear_session()
    tr_ind, val_ind = splits[fold]
    model = build_model(embedding_matrix)
    model.fit(x_train[tr_ind],
        y_train[tr_ind]>0.5,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
        validation_data=(x_train[val_ind], y_train[val_ind]>0.5))
    
    train_preds[val_ind] += model.predict(x_train[val_ind])[:,0]
    test_preds += model.predict(x_test)[:,0]
test_preds /= 4

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1353655 samples, validate on 451219 samples
Epoch 1/2
1353655/1353655 [==============================] - 561s 415us/step - loss: 0.1202 - acc: 0.9567 - val_loss: 0.0925 - val_acc: 0.9646
Epoch 2/2
1353655/1353655 [==============================] - 557s 412us/step - loss: 0.0921 - acc: 0.9645 - val_loss: 0.0872 - val_acc: 0.9664
Train on 1353655 samples, validate on 451219 samples
Epoch 1/2
1353655/1353655 [==============================] - 557s 412us/step - loss: 0.1180 - acc: 0.9581 - val_loss: 0.0877 - val_acc: 0.9661
Epoch 2/2
1353655/1353655 [==============================] - 558s 413us/step - loss: 0.0929 - acc: 0.9642 - val_loss: 0.0833 - val_acc: 0.9675
Train on 

In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train>0.5, train_preds)

0.9720556970915125

In [16]:
df_submit = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
df_submit.prediction = test_preds
df_submit.to_csv('submission.csv', index=False)

* the base code of attention is from the kernel below [https://www.kaggle.com/christofhenkel/keras-baseline-lstm-attention-5-fold]